# Forward Simulation of the Acoustic Wave Equation Using PINNs

In this notebook we solve the acoustic wave equation

$$
u_{tt}(x,t) = c^2 \; u_{xx}(x,t),
$$

where $u(x,t)$ is the acoustic pressure (or displacement) field, $c$ is the wave speed, and $u_{tt}$ and $u_{xx}$ denote the second derivatives with respect to time and space, respectively.

We will solve this PDE on a rectangular domain:

- Spatial domain: $x \in [0, 1]$
- Time domain: $t \in [0, 1]$

with the following conditions:

- **Initial conditions:**
  - $u(x, 0) = \sin(\pi x)$
  - $u_t(x, 0) = 0$
- **Boundary conditions:**
  - $u(0, t) = 0$ and $u(1, t) = 0$

The goal is to use a Physics-Informed Neural Network (PINN) to approximate the solution \(u(x,t)\) by minimizing a loss function that includes the residual of the PDE, the initial conditions, and the boundary conditions.

## Import Libraries

We begin by importing DeepXDE, NumPy, Matplotlib, and other necessary libraries. DeepXDE uses automatic differentiation to compute derivatives needed to enforce the PDE.

In [ ]:
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt
from deepxde.backend import tf

## Governing Equation

The acoustic wave equation is given by:

$$
u_{tt}(x,t) - c^2 \; u_{xx}(x,t) = 0,
$$

where $c$ is the wave speed (we set, for example, $c=1$ for simplicity). The PINN will approximate $u(x,t)$ such that the residual of this equation is minimized over the domain.

## Define the PDE Residual Function

We define a function that computes the residual of the PDE. Here, we use DeepXDE's automatic differentiation to compute $u_{tt}$ and $u_{xx}$.

In [ ]:
def acoustic_wave_pde(x, u):
    """
    Returns the residual of the acoustic wave equation:
      u_tt - c^2 * u_xx
    where u(x,t) is the solution.
    """
    # Compute second derivative with respect to time
    u_t = dde.grad.jacobian(u, x, i=1)  # derivative with respect to t (x[:,1])
    u_tt = dde.grad.jacobian(u_t, x, i=1)

    # Compute second derivative with respect to space
    u_x = dde.grad.jacobian(u, x, i=0)  # derivative with respect to x (x[:,0])
    u_xx = dde.grad.jacobian(u_x, x, i=0)

    c = 1.0  # wave speed
    return u_tt - c**2 * u_xx

## Define the Domain, Initial, and Boundary Conditions

We define a rectangular geometry for the domain:

- Spatial variable $x \in [0, 1]$
- Time variable $t \in [0, 1]$

The initial conditions are set as follows:

- $u(x,0) = \sin(\pi x)$
- $u_t(x,0) = 0$

The boundary conditions are homogeneous Dirichlet:

- $u(0,t) = 0$ and $u(1,t) = 0$

In [ ]:
# Define the geometry: a rectangle in the (x,t) space
geom = dde.geometry.Rectangle(xmin=[0, 0], xmax=[1, 1])

# Initial condition: u(x,0) = sin(pi*x)
def func_u_init(x):
    return np.sin(np.pi * x[:, 0:1])

# Initial condition for the time derivative: u_t(x,0) = 0
def func_ut_init(x):
    return np.zeros((x.shape[0], 1))

# Define the initial condition (IC) for u at t = 0
ic_u = dde.icbc.IC(geom, func_u_init, lambda x, on_initial: on_initial, component=0)

# Define the initial condition for the time derivative
ic_ut = dde.icbc.IC(geom, func_ut_init, lambda x, on_initial: on_initial, component=1)

# For the PDE, we need to supply the initial conditions in the data formulation.
# Note: In this formulation, we can combine the initial and boundary conditions using the built-in DeepXDE classes.

# Boundary condition: u(0,t) = 0 and u(1,t) = 0
bc_left = dde.icbc.DirichletBC(geom, lambda x: 0, lambda x, on_boundary: np.isclose(x[0], 0))
bc_right = dde.icbc.DirichletBC(geom, lambda x: 0, lambda x, on_boundary: np.isclose(x[0], 1))

# Combine all boundary and initial conditions
data = dde.data.TimePDE(
    geom,
    acoustic_wave_pde,
    [ic_u, ic_ut, bc_left, bc_right],
    num_domain=20000,
    num_boundary=2000
)

## Setup and Train the PINN Model

We use a feed-forward neural network (FNN) with 4 hidden layers (50 neurons each) to approximate the solution $u(x,t)$. The network takes a two-dimensional input $[x,t]$ and outputs a single scalar ($u$). An output transform is applied to help enforce the initial condition. Finally, the model is compiled with the Adam optimizer and trained for 50,000 iterations.

In [ ]:
layer_size = [2] + [50] * 4 + [1]  # Input: [x, t]; Output: u
activation = "tanh"
initializer = "Glorot normal"
net = dde.nn.FNN(layer_size, activation, initializer)

def output_transform(x, u):
    """
    An output transform that enforces the initial condition u(x,0) = sin(pi*x).
    Here we add the known initial condition and multiply the network output by t to ensure u(x,0) is satisfied.
    """
    t = x[:, 1:2]
    return u * t + np.sin(np.pi * x[:, 0:1])

net.apply_output_transform(output_transform)

model = dde.Model(data, net)
model.compile("adam", lr=0.001)


# Create output directory for saving checkpoints
path = "./../output/Model/model"
os.makedirs(path, exist_ok=True)
checkpoint_path = os.path.join(path, "model.ckpt")
checker = dde.callbacks.ModelCheckpoint(
      checkpoint_path, save_better_only=True, period=50
  )

# Train the model for 50,000 iterations (epochs)
losshistory, train_state = model.train(epochs=50000, callbacks=[checker])

## Prediction and Plotting

After training, we predict the solution $u(x,t)$ on a fine grid over the domain and plot the results.

Here we plot both the solution at a fixed time (e.g., $t=0.5$) as a function of $x$ and a surface plot in the $(x,t)$ space.

In [ ]:
# Create a grid for prediction
x = np.linspace(0, 1, 200)[:, None]
t = np.linspace(0, 1, 200)[:, None]
X, T = np.meshgrid(x, t)
XT = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))

# Predict the solution
u_pred = model.predict(XT)
u_pred = u_pred.reshape(X.shape)

# Plot the solution at a fixed time t = 0.5
plt.figure(figsize=(8, 6))
plt.plot(x, u_pred[100, :], 'r-', label='Predicted u(x,t=0.5)')
plt.xlabel('x')
plt.ylabel('u')
plt.legend()
plt.title('Solution at t = 0.5')
plt.grid(True)
plt.show()

# Surface plot of u(x,t)
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, T, u_pred, cmap='viridis', edgecolor='none')
ax.set_xlabel('x')
ax.set_ylabel('t')
ax.set_zlabel('u(x,t)')
ax.set_title('Acoustic Wave Simulation')
plt.show()